# Reddit dataset

In [3]:
import numpy as np
import pandas as pd
import sqlite3 as sql

save_path = "C:/Users\TS/PycharmProjects/DS1-RecommendationSystems/src/data/reddit/"

In [4]:
db = sql.connect("T:/SubredditDataset/data_original.db")

query = "SELECT * FROM comment"
df = pd.read_sql_query(query, db)
print(f"DF size: {df.shape}\nDF columns: {list(df.columns)}")

DF size: (20000571, 10)nDF columns: ['ID', 'user', 'subreddit', 'content', 'num_upvotes', 'num_downvotes', 'num_comments', 'created_time_utc', 'link_title', 'link_id']


TODO:
1. Duplicate entries will be removed.
* The table comment contains a lot of information we dont need for the recommender such as the number of upvotes.
* If a user has written multiple comments in a subreddit, there is one data point for each in the table.
 For this reason, all comments from a user in the same subreddit are combined and a count variable is introduced.

In [5]:
# Remove duplicates given by a subset of the columns
df_no_duplicates = df.drop_duplicates(keep='first',
                                      ignore_index=True,
                                      subset=["user", "subreddit", "created_time_utc", "link_id"])
# We only need the user and the subreddit
df_no_duplicates = df_no_duplicates[['user', 'subreddit']]
print(f"DF size: {df_no_duplicates.shape}\nDF columns: {list(df_no_duplicates.columns)}")

DF size: (20000571, 10)nDF columns: ['ID', 'user', 'subreddit', 'content', 'num_upvotes', 'num_downvotes', 'num_comments', 'created_time_utc', 'link_title', 'link_id']


In [64]:
df_grouped = df_no_duplicates.groupby(by=['user', 'subreddit']).size().reset_index(name='count')
df_grouped.to_csv(save_path + "dataset.csv", index=False)
df_grouped.describe(include="all")

,user,subreddit,count
count,1738737,1738737,1.738737e+06
unique,37845,69490,NaN
top,CarpenterAcademic,AskReddit,NaN
freq,851,21021,NaN
mean,NaN,NaN,1.121939e+01
std,NaN,NaN,4.808560e+01
min,NaN,NaN,1.000000e+00
25%,NaN,NaN,1.000000e+00
50%,NaN,NaN,2.000000e+00
75%,NaN,NaN,6.000000e+00


In [62]:
subreddit_group = df_grouped.groupby(by=['subreddit'])['count']
sum_comments_per_subreddit = subreddit_group.sum().reset_index(name="total_num_comments")
sum_comments_per_subreddit = sum_comments_per_subreddit.sort_values(by="total_num_comments", ascending=False,
                                                                    ignore_index=True)
unique_users_per_subreddit = subreddit_group.count().reset_index(name="unique_users")
subreddit_summary = sum_comments_per_subreddit.merge(unique_users_per_subreddit, on="subreddit")
subreddit_summary.to_csv(save_path + "subreddit_summary.csv", index=False)

In [63]:
user_group = df_grouped.groupby(by=['user'])['count']
sum_comments_per_user = user_group.sum().reset_index(name="total_num_comments")
sum_comments_per_user = sum_comments_per_user.sort_values(by="total_num_comments", ascending=False, ignore_index=True)
unique_subreddits_per_user = user_group.count().reset_index(name="unique_subreddits")
user_summary = sum_comments_per_user.merge(unique_subreddits_per_user, on="user")
user_summary.to_csv(save_path + "user_summary.csv", index=False)


In [5]:
subreddit_info = pd.read_csv(save_path + "subreddit_info.csv")
subreddit_info = subreddit_info[['subreddit', 'num_subscribers', 'over18', 'public_description']]

In [6]:
subreddit_info_clean = subreddit_info.dropna(axis=0, how="any")
print(subreddit_info.shape[0] - subreddit_info_clean.shape[0])

0


In [7]:
subreddit_info_clean.over18 = subreddit_info_clean.over18.astype('bool')

In [9]:
subreddit_info_clean.to_csv(save_path + "subreddit_info.csv", index=False)

In [8]:
subreddit_info_clean.head()


,subreddit,num_subscribers,over18,public_description
0,ChoosingBeggars,2134849.0,False,"This subreddit is for posting screenshots, pic..."
1,Python,809272.0,False,News about the programming language Python. If...
2,interestingasfuck,8092462.0,False,For anything that is InterestingAsFuck
3,PublicFreakout,3257059.0,False,"A subreddit dedicated to people freaking out, ..."
4,ShitMomGroupsSay,258681.0,False,Share the drama. \n\nEssential oils cure all? ...
